# SNMP - Simple Network Management Protocol

## Install snmpd tools, agent and mibs

In [76]:
%%bash

sudo apt update
sudo apt install -y snmp snmpd snmp-mibs-downloader

Hit:1 https://dl.yarnpkg.com/debian stable InRelease
Hit:2 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease
Hit:3 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease


Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [3722 kB]
Hit:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1515 kB]
Hit:7 https://packagecloud.io/github/git-lfs/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [4223 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1230 kB]
Get:13 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [3751 kB]
Get:14 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [3877 kB]
Fetched 18.6 MB in 2s (9107 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
29 packages can be up

Reading package lists...
Building dependency tree...
Reading state information...
snmp-mibs-downloader is already the newest version (1.2).
snmp is already the newest version (5.8+dfsg-2ubuntu2.9).
snmpd is already the newest version (5.8+dfsg-2ubuntu2.9).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


## Configure agent configuration file (/etc/snmp/snmpd.conf) and snmp tools configuration file (/etc/snmp/snmp.conf)

In [1]:
%%bash

sudo mv /etc/snmp/snmpd.conf /etc/snmp/snmpd.conf.old

sudo tee /etc/snmp/snmpd.conf > /dev/null <<EOF
rocommunity public
rwcommunity private
EOF

sudo sed -i '/mibs :/s/^/#/' /etc/snmp/snmp.conf

## Start snmpd daemon (agent)

In [2]:
%%bash

sudo service snmpd restart
sudo service snmpd status

 * snmpd is running


## List all downloaded mibs

In [3]:
%%bash

ls -R /usr/share/snmp/mibs

/usr/share/snmp/mibs:
AIRCONDITIONER-MIB.txt
GNOME-SMI.txt
LM-SENSORS-MIB.txt
NET-SNMP-AGENT-MIB.txt
NET-SNMP-EXAMPLES-MIB.txt
NET-SNMP-EXTEND-MIB.txt
NET-SNMP-MIB.txt
NET-SNMP-MONITOR-MIB.txt
NET-SNMP-PASS-MIB.txt
NET-SNMP-PERIODIC-NOTIFY-MIB.txt
NET-SNMP-SYSTEM-MIB.txt
NET-SNMP-TC.txt
NET-SNMP-VACM-MIB.txt
UCD-DEMO-MIB.txt
UCD-DISKIO-MIB.txt
UCD-DLMOD-MIB.txt
UCD-IPFILTER-MIB.txt
UCD-IPFWACC-MIB.txt
UCD-SNMP-MIB-OLD.txt
UCD-SNMP-MIB.txt
iana
ietf


## Show RFC1213 object tree

In [252]:
%%bash

# Variables
MIB="/usr/share/snmp/mibs/ietf/RFC1213-MIB"

#snmptranslate -m $MIB -Tp

cat $MIB

RFC1213-MIB DEFINITIONS ::= BEGIN

IMPORTS
        mgmt, NetworkAddress, IpAddress, Counter, Gauge,
                TimeTicks
            FROM RFC1155-SMI
        OBJECT-TYPE
                FROM RFC-1212;

--  This MIB module uses the extended OBJECT-TYPE macro as
--  defined in [14];

--  MIB-II (same prefix as MIB-I)

mib-2      OBJECT IDENTIFIER ::= { mgmt 1 }

-- textual conventions

DisplayString ::=
    OCTET STRING
-- This data type is used to model textual information taken
-- from the NVT ASCII character set.  By convention, objects
-- with this syntax are declared as having

--
--      SIZE (0..255)

PhysAddress ::=
    OCTET STRING
-- This data type is used to model media addresses.  For many
-- types of media, this will be in a binary representation.
-- For example, an ethernet address would be represented as
-- a string of 6 octets.

-- groups in MIB-II

system       OBJECT IDENTIFIER ::= { mib-2 1 }

interfaces   OBJECT IDENTIFIER ::= { mib-2 2 }

at           OBJECT IDE

## Show the description of a specific object

In [23]:
%%bash

# Variables
OBJECT="sysDescr"
MIB="/usr/share/snmp/mibs/ietf/RFC1213-MIB"

# Command to extract text from search_string to } excluding the }
sed -n "/$OBJECT OBJECT-TYPE/,/}/p" $MIB

sysDescr OBJECT-TYPE
    SYNTAX  DisplayString (SIZE (0..255))
    ACCESS  read-only
    STATUS  mandatory
    DESCRIPTION
            "A textual description of the entity.  This value
            should include the full name and version
            identification of the system's hardware type,
            software operating-system, and networking
            software.  It is mandatory that this only contain
            printable ASCII characters."
    ::= { system 1 }


## Using snmpget

In [24]:
%%bash

# snmpget -v <SNMP version> -c <community> <host> <obj1> <obj2> <obj3> 

snmpget -v 1 -c public localhost sysDescr.0 sysContact.0

SNMPv2-MIB::sysDescr.0 = STRING: Linux codespaces-396979 6.5.0-1021-azure #22~22.04.1-Ubuntu SMP Tue Apr 30 16:08:18 UTC 2024 x86_64
SNMPv2-MIB::sysContact.0 = STRING: admin@localhost


## Using snmpset

In [25]:
%%bash

snmpset -v 1 -c private localhost sysContact.0 s admin@localhost
snmpget -v 1 -c public localhost sysContact.0

SNMPv2-MIB::sysContact.0 = STRING: admin@localhost


SNMPv2-MIB::sysContact.0 = STRING: admin@localhost


## Using snmpgetnext

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost system interfaces

## Using snmpwalk

In [ ]:
%%bash

# run on terminal: sudo tcpdump -i lo -n port 161 -v
snmpwalk -v 1 -c public localhost system

## Other commands

In [ ]:
%%bash

snmptable -v 1 -c public localhost ifTable

In [ ]:
%%bash

snmpdelta -v 1 -c public -Cs -CT localhost sysUpTime.0

In [ ]:
%%bash

killall snmpdelta

## Exercícios - SNMPv1

a) obter o endereço físico (MAC) da 2a. interface de rede da tabela ifTable

In [ ]:
%%bash

snmpget -v 1 -c public localhost ifPhysAddress.2

b) obter o número de mensagens ICMP enviadas e recebidas pelo elemento gerenciado

In [ ]:
%%bash

snmpget -v 1 -c public localhost icmpInMsgs.0 icmpOutMsgs.0

c) obter o número de requisições GET recebidos pelo agente no elemento gerenciado.

In [ ]:
%%bash

snmpget -v 1 -c public localhost snmpInGetRequests.0

d) modificar o nome do elemento gerenciado para "Gerencia"

In [ ]:
%%bash

snmpset -v 1 -c private localhost sysName.0 s "Gerencia"

e) modificar a identificação da localização do elemento gerenciado para "LabRedes"

In [ ]:
%%bash

snmpset -v 1 -c private localhost sysLocation.0 s "LabRedes"

f) obter a descrição da interface localizada na 1a. linha da tabela ifTable

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost ifDescr

g) obter o endereço IP, índice da interface na tabela ifTable, máscara de rede e endereço de broadcast da primeira entrada da tabela ipAddrEntry

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost ipAdEntIfIndex ipAdEntNetMask ipAdEntBcastAddr

h) obter o endereço local e porta local da primeira linha da tabela udpTable

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost udpLocalPort udpLocalAddress

## Using snmpbulkget

In [ ]:
%%bash

snmpbulkget -v2c -c public localhost -Cn2 -Cr3 system interfaces ifIndex ifDescr ifSpeed

In [ ]:
%%bash

snmpbulkget -v2c -c public localhost -Cn4 -Cr5 tcpInSegs tcpOutSegs tcpRetransSegs tcpInErrs tcpConnState tcpConnLocalAddress tcpConnLocalPort tcpConnRemAddress tcpConnRemPort

In [ ]:
%%bash

snmpbulkget -v2c -c public localhost -Cn0 -Cr4 udpLocalAddress udpLocalPort

## Comparing snmpwalk vs snmpbulkwalk



In [ ]:
%%bash

# run on terminal: sudo tcpdump -i lo -n port 161 -v
# snmpwalk -v 1 -c public localhost system

snmpbulkwalk -v 2c -c public localhost system

# Example using pass directive

## snmpd.conf

In [12]:
%%bash

# Add pass directive to snmpd.conf
sudo tee /etc/snmp/snmpd.conf > /dev/null <<EOF
rocommunity public
rwcommunity private

pass .1.3.6.1.3.99999.1 /usr/bin/python3 /tmp/agent.py
EOF

# Restart agent
sudo service snmpd restart
sudo service snmpd status

 * snmpd is running


## Agent code

In [97]:
%%writefile /tmp/agent.py

#!/usr/bin/env python3

import sys
import datetime
import socket

# Variáveis para simular o estado do ar condicionado
ac_status = 1  # 0: off, 1: on
ac_current_temperature = 22  # Temperatura atual em Celsius
ac_desired_temperature = 24  # Temperatura desejada em Celsius
ac_mode = 2  # 1: auto, 2: cool, 3: heat, 4: fan
ac_fan_speed = 3  # 1: low, 2: medium, 3: high
ac_error_message = "errooooo"  # Mensagem de erro, se houver



def get_ac_status():
    return ac_status

def get_ac_current_temperature():
    return ac_current_temperature

def get_ac_desired_temperature():
    return ac_desired_temperature

def get_ac_mode():
    return ac_mode

def get_ac_fan_speed():
    return ac_fan_speed

def get_ac_error_message():
    return ac_error_message

def set_ac_desired_temperature(value):
    global ac_desired_temperature
    ac_desired_temperature = value
    return ac_desired_temperature

def main():

    with open("/tmp/agent.log", 'w') as file:
        file.write(' '.join(sys.argv))

    if len(sys.argv) != 3:
        print("Usage: agent.py <MIB-oid> <request-type>")
        return

    request_type = sys.argv[1]
    oid = sys.argv[2]

    if request_type == "-g":  # GET request
        if oid == ".1.3.6.1.3.99999.1.1.1":
            print(".1.3.6.1.3.99999.1.1.1")
            print("integer")
            print(get_ac_status())
        elif oid == ".1.3.6.1.3.99999.1.1.2":
            print(".1.3.6.1.3.99999.1.1.1.2")
            print("integer")
            print(get_ac_current_temperature())
        elif oid == ".1.3.6.1.3.99999.1.1.3":
            print(".1.3.6.1.3.99999.1.1.3")
            print("integer")
            print(get_ac_desired_temperature())
        elif oid == ".1.3.6.1.3.99999.1.1.4":
            print(".1.3.6.1.4.1.99999.1.1.4")
            print("integer")
            print(get_ac_mode())
        elif oid == ".1.3.6.1.3.99999.1.1.5":
            print(".1.3.6.1.4.1.99999.1.1.5")
            print("integer")
            print(get_ac_fan_speed())
        elif oid == ".1.3.6.1.3.99999.1.1.6":
            print(".1.3.6.1.4.1.99999.1.1.6")
            print("string")
            print(get_ac_error_message())
        else:
            print("NONE")
    elif request_type == "-s":  # SET request
        print("aquiiii")
        if oid == ".1.3.6.1.3.99999.1.1.3":
            try:
                value = int(sys.argv[4])
                result = set_ac_desired_temperature(value)
                print(".1.3.6.1.4.1.99999.1.1.3")
                print("integer")
                print(result)
            except ValueError:
                print("Invalid value")
        else:
            print("NONE")
    else:
        print("NONE")

if __name__ == "__main__":
    main()



Overwriting /tmp/agent.py


In [143]:
%%writefile /tmp/agent.py

#!/usr/bin/env python3

import sys
import os
import tempfile

# Definindo o diretório de dados no diretório temporário do sistema
data_dir = '/tmp'

# Caminhos dos arquivos para cada variável
files = {
    "ac_status": os.path.join(data_dir, 'ac_status.txt'),
    "ac_current_temperature": os.path.join(data_dir, 'ac_current_temperature.txt'),
    "ac_desired_temperature": os.path.join(data_dir, 'ac_desired_temperature.txt'),
    "ac_mode": os.path.join(data_dir, 'ac_mode.txt'),
    "ac_fan_speed": os.path.join(data_dir, 'ac_fan_speed.txt'),
    "ac_error_message": os.path.join(data_dir, 'ac_error_message.txt')
}

# Funções para ler e escrever valores em arquivos
def read_value(filename):
    try:
        with open(filename, 'r') as file:
            return file.readline().strip()
    except FileNotFoundError:
        return None

def write_value(filename, value):
    with open(filename, 'w') as file:
        file.write(str(value))

# Inicializando os valores ao carregar o script
ac_status = int(read_value(files["ac_status"]) or 1)
ac_current_temperature = int(read_value(files["ac_current_temperature"]) or 37)
ac_desired_temperature = int(read_value(files["ac_desired_temperature"]) or 55)
ac_mode = int(read_value(files["ac_mode"]) or 2)
ac_fan_speed = int(read_value(files["ac_fan_speed"]) or 3)
ac_error_message = read_value(files["ac_error_message"]) or "errooooo"

def get_ac_status():
    return ac_status

def get_ac_current_temperature():
    return ac_current_temperature

def get_ac_desired_temperature():
    return ac_desired_temperature

def get_ac_mode():
    return ac_mode

def get_ac_fan_speed():
    return ac_fan_speed

def get_ac_error_message():
    return ac_error_message

def set_ac_desired_temperature(value):
    global ac_desired_temperature
    ac_desired_temperature = value
    write_value(files["ac_desired_temperature_abacate"], ac_desired_temperature)
    return ac_desired_temperature

def main():

    with open("/tmp/agent.log", 'a') as file:
        file.write(' '.join(sys.argv))

    if len(sys.argv) != 3 and len(sys.argv) != 5:
        print("Usage: agent.py <MIB-oid> <request-type> [<type> <value>]")
        return

    request_type = sys.argv[1]
    oid = sys.argv[2]

    if request_type == "-g":  # GET request
        if oid == ".1.3.6.1.3.99999.1.1.1":
            print(".1.3.6.1.3.99999.1.1.1")
            print("integer")
            print(get_ac_status())
        elif oid == ".1.3.6.1.3.99999.1.1.2":
            print(".1.3.6.1.3.99999.1.1.2")
            print("integer")
            print(get_ac_current_temperature())
        elif oid == ".1.3.6.1.3.99999.1.1.3":
            print(".1.3.6.1.3.99999.1.1.3")
            print("integer")
            print(get_ac_desired_temperature())
        elif oid == ".1.3.6.1.3.99999.1.1.4":
            print(".1.3.6.1.3.99999.1.1.4")
            print("integer")
            print(get_ac_mode())
        elif oid == ".1.3.6.1.3.99999.1.1.5":
            print(".1.3.6.1.3.99999.1.1.5")
            print("integer")
            print(get_ac_fan_speed())
        elif oid == ".1.3.6.1.3.99999.1.1.6":
            print(".1.3.6.1.3.99999.1.1.6")
            print("string")
            print(get_ac_error_message())
        else:
            print("NONE")
    elif request_type == "-s":  # SET request
        if oid == ".1.3.6.1.3.99999.1.1.3" and len(sys.argv) == 5:
            try:
                value = int(sys.argv[4])
                result = set_ac_desired_temperature(value)
                print(".1.3.6.1.3.99999.1.1.3")
                print("integer")
                print(result)
            except ValueError:
                print("Invalid value")
        else:
            print("NONE")
    else:
        print("NONE")

if __name__ == "__main__":
    main()


Overwriting /tmp/agent.py


In [160]:
%%writefile /tmp/agent.py

#!/usr/bin/env python3

import sys
import os

# Definindo o diretório de dados na pasta /tmp
data_dir = '/tmp'

# Caminhos dos arquivos para cada variável
files = {
    "ac_status": os.path.join(data_dir, 'ac_status.txt'),
    "ac_current_temperature": os.path.join(data_dir, 'ac_current_temperature.txt'),
    "ac_desired_temperature": os.path.join(data_dir, 'ac_desired_temperature.txt'),
    "ac_mode": os.path.join(data_dir, 'ac_mode.txt'),
    "ac_fan_speed": os.path.join(data_dir, 'ac_fan_speed.txt'),
    "ac_error_message": os.path.join(data_dir, 'ac_error_message.txt')
}

# Funções para ler e escrever valores em arquivos
def read_value(filename):
    try:
        with open(filename, 'r') as file:
            return file.readline().strip()
    except FileNotFoundError:
        return None

def write_value(filename, value):
    print("aqui yo to")
    try:
        with open(filename, 'w') as file:
            file.write(str(value))
        print(f"Successfully wrote {value} to {filename}")
    except Exception as e:
        print(f"Error writing to {filename}: {e}")

# Inicializando os valores ao carregar o script
ac_status = int(read_value(files["ac_status"]) or 1)
ac_current_temperature = int(read_value(files["ac_current_temperature"]) or 22)
ac_desired_temperature = int(read_value(files["ac_desired_temperature"]) or 24)
ac_mode = int(read_value(files["ac_mode"]) or 2)
ac_fan_speed = int(read_value(files["ac_fan_speed"]) or 3)
ac_error_message = read_value(files["ac_error_message"]) or "errooooo"

def get_ac_status():
    return ac_status

def get_ac_current_temperature():
    return ac_current_temperature

def get_ac_desired_temperature():
    return ac_desired_temperature

def get_ac_mode():
    return ac_mode

def get_ac_fan_speed():
    return ac_fan_speed

def get_ac_error_message():
    return ac_error_message

def set_ac_desired_temperature(value):
    global ac_desired_temperature
    ac_desired_temperature = value
    print("estoy aqui")
    write_value(files["ac_desired_temperature"], ac_desired_temperature)
    return ac_desired_temperature

def main():

    with open("/tmp/agent.log", 'a') as file:
        file.write(' '.join(sys.argv) + '\n')

    if len(sys.argv) != 3 and len(sys.argv) != 5:
        print("Usage: agent.py <MIB-oid> <request-type> [<type> <value>]")
        return

    request_type = sys.argv[1]
    oid = sys.argv[2]

    if request_type == "-g":  # GET request
        if oid == ".1.3.6.1.3.99999.1.1.1":
            print(".1.3.6.1.3.99999.1.1.1")
            print("integers")
            print(get_ac_status())
        elif oid == ".1.3.6.1.3.99999.1.1.2":
            print(".1.3.6.1.3.99999.1.1.2")
            print("integerw")
            print(get_ac_current_temperature())
        elif oid == ".1.3.6.1.3.99999.1.1.3":
            print(".1.3.6.1.3.99999.1.1.3")
            print("integerq")
            print(get_ac_desired_temperature())
        elif oid == ".1.3.6.1.3.99999.1.1.4":
            print(".1.3.6.1.3.99999.1.1.4")
            print("integerk")
            print(get_ac_mode())
        elif oid == ".1.3.6.1.3.99999.1.1.5":
            print(".1.3.6.1.3.99999.1.1.5")
            print("integerh")
            print(get_ac_fan_speed())
        elif oid == ".1.3.6.1.3.99999.1.1.6":
            print(".1.3.6.1.3.99999.1.1.6")
            print("string tgg")
            print(get_ac_error_message())
        else:
            print("NONE")
    elif request_type == "-s":  # SET request
        if oid == ".1.3.6.1.3.99999.1.1.3" and len(sys.argv) == 5:
            try:
                type = sys.argv[3]
                value = int(sys.argv[4])
                #if type == "i":
                    result = set_ac_desired_temperature(value)
                    print(".1.3.6.1.3.99999.1.1.3")
                    print("integerrrr")
                    print(result)
                #else:
                    print(f"Unsupported type: {type}")
            except ValueError:
                print("Invalid value")
        else:
            print("NONE")
    else:
        print("NONE")

if __name__ == "__main__":
    main()


Overwriting /tmp/agent.py


## Testing

In [3]:
%%bash

cd /tmp/
echo 23 > ac_current_temperature.txt

echo 32 > ac_desired_temperature.txt
echo 1 > ac_mode.txt
echo 2 > ac_fan_speed.txt
echo explodiu > ac_error_message.txt

chmod +777 ac_current_temperature.txt

chmod +777 ac_desired_temperature.txt
chmod +777 ac_mode.txt
chmod +777 ac_fan_speed.txt
chmod +777 ac_error_message.txt

In [179]:
%%writefile /tmp/agent.py

#!/usr/bin/env python3

import sys
import os

# Definindo o diretório de dados na pasta /tmp
data_dir = '/tmp'

# Caminhos dos arquivos para cada variável
files = {
    "ac_status": os.path.join(data_dir, 'ac_status.txt'),
    "ac_current_temperature": os.path.join(data_dir, 'ac_current_temperature.txt'),
    "ac_desired_temperature": os.path.join(data_dir, 'ac_desired_temperature.txt'),
    "ac_mode": os.path.join(data_dir, 'ac_mode.txt'),
    "ac_fan_speed": os.path.join(data_dir, 'ac_fan_speed.txt'),
    "ac_error_message": os.path.join(data_dir, 'ac_error_message.txt')
}

# Funções para ler e escrever valores em arquivos
def read_value(filename):
    try:
        with open(filename, 'r') as file:
            return file.readline().strip()
    except FileNotFoundError:
        return None

def write_value(filename, value):
    try:
        with open(filename, 'w') as file:
            file.write(str(value))
        log(f"Successfully wrote {value} to {filename}")
    except Exception as e:
        log(f"Error writing to {filename}: {e}")

# Função de log para escrever mensagens no arquivo de log
def log(message):
    with open("/tmp/agent_output.log", 'a') as log_file:
        log_file.write(message + '\n')

# Inicializando os valores ao carregar o script
ac_status = int(read_value(files["ac_status"]) or 1)
ac_current_temperature = int(read_value(files["ac_current_temperature"]) or 22)
ac_desired_temperature = int(read_value(files["ac_desired_temperature"]) or 24)
ac_mode = int(read_value(files["ac_mode"]) or 2)
ac_fan_speed = int(read_value(files["ac_fan_speed"]) or 3)
ac_error_message = read_value(files["ac_error_message"]) or "errooooo"

def get_ac_status():
    return ac_status

def get_ac_current_temperature():
    return ac_current_temperature

def get_ac_desired_temperature():
    return ac_desired_temperature

def get_ac_mode():
    return ac_mode

def get_ac_fan_speed():
    return ac_fan_speed

def get_ac_error_message():
    return ac_error_message

def set_ac_desired_temperature(value):
    global ac_desired_temperature
    ac_desired_temperature = value
    write_value(files["ac_desired_temperature"], ac_desired_temperature)
    return ac_desired_temperature

def main():

    with open("/tmp/agent.log", 'a') as file:
        file.write(' '.join(sys.argv) + '\n')

    if len(sys.argv) != 3 and len(sys.argv) != 5:
        log("Usage: agent.py <MIB-oid> <request-type> [<type> <value>]")
        return

    request_type = sys.argv[1]
    oid = sys.argv[2]

    if request_type == "-g":  # GET request
        if oid == ".1.3.6.1.3.99999.1.1.1":
            log(f"GET request for ac_status: {get_ac_status()}")
            print(".1.3.6.1.3.99999.1.1.1")
            print("integer")
            print(get_ac_status())
        elif oid == ".1.3.6.1.3.99999.1.1.2":
            log(f"GET request for ac_current_temperature: {get_ac_current_temperature()}")
            print(".1.3.6.1.3.99999.1.1.2")
            print("integer")
            print(get_ac_current_temperature())
        elif oid == ".1.3.6.1.3.99999.1.1.3":
            log(f"GET request for ac_desired_temperature: {get_ac_desired_temperature()}")
            print(".1.3.6.1.3.99999.1.1.3")
            print("integer")
            print(get_ac_desired_temperature())
        elif oid == ".1.3.6.1.3.99999.1.1.4":
            log(f"GET request for ac_mode: {get_ac_mode()}")
            print(".1.3.6.1.3.99999.1.1.4")
            print("integer")
            print(get_ac_mode())
        elif oid == ".1.3.6.1.3.99999.1.1.5":
            log(f"GET request for ac_fan_speed: {get_ac_fan_speed()}")
            print(".1.3.6.1.3.99999.1.1.5")
            print("integer")
            print(get_ac_fan_speed())
        elif oid == ".1.3.6.1.3.99999.1.1.6":
            log(f"GET request for ac_error_message: {get_ac_error_message()}")
            print(".1.3.6.1.3.99999.1.1.6")
            print("string")
            print(get_ac_error_message())
        else:
            log("Invalid OID for GET request")
            print("NONE")
    elif request_type == "-s":  # SET request
        if oid == ".1.3.6.1.3.99999.1.1.3" and len(sys.argv) == 5:
            try:
                type = sys.argv[3]
                value = int(sys.argv[4])
                if type == "i":
                    result = set_ac_desired_temperature(value)
                    log(f"SET request for ac_desired_temperature: {result}")
                    print(".1.3.6.1.3.99999.1.1.3")
                    print("integer")
                    print("tipo",type)
                    print(result)
                else:
                    log(f"Unsupported type for SET request: {type}")
                    print(f"Unsupported type: {type}")
            except ValueError:
                log("Invalid value for SET request")
                print("Invalid value")
        else:
            log("Invalid OID or arguments for SET request")
            print("NONE")
    else:
        log("Invalid request type")
        print("NONE")

if __name__ == "__main__":
    main()


Overwriting /tmp/agent.py


Set Working
|
|
|
|
|
V

In [348]:
%%writefile /tmp/agent.py

#!/usr/bin/env python3

import sys
import os

# Definindo o diretório de dados na pasta /tmp
data_dir = '/tmp'

# Caminhos dos arquivos para cada variável
files = {
    "ac_status": os.path.join(data_dir, 'ac_status.txt'),
    "ac_current_temperature": os.path.join(data_dir, 'ac_current_temperature.txt'),
    "ac_desired_temperature": os.path.join(data_dir, 'ac_desired_temperature.txt'),
    "ac_mode": os.path.join(data_dir, 'ac_mode.txt'),
    "ac_fan_speed": os.path.join(data_dir, 'ac_fan_speed.txt'),
    "ac_error_message": os.path.join(data_dir, 'ac_error_message.txt')
}

# Funções para ler e escrever valores em arquivos
def read_value(filename):
    try:
        with open(filename, 'r') as file:
            return file.readline().strip()
    except FileNotFoundError:
        return None

def write_value(filename, value):
    try:
        with open(filename, 'w') as file:
            file.write(str(value))
        log(f"Successfully wrote {value} to {filename}")
    except Exception as e:
        log(f"Error writing to {filename}: {e}")

# Função de log para escrever mensagens no arquivo de log
def log(message):
    with open("/tmp/agent_output.log", 'a') as log_file:
        log_file.write(message + '\n')

# Inicializando os valores ao carregar o script
ac_status = int(read_value(files["ac_status"]) or 1)
ac_current_temperature = int(read_value(files["ac_current_temperature"]) or 22)
ac_desired_temperature = int(read_value(files["ac_desired_temperature"]) or 24)
ac_mode = int(read_value(files["ac_mode"]) or 2)
ac_fan_speed = int(read_value(files["ac_fan_speed"]) or 3)
ac_error_message = read_value(files["ac_error_message"]) or "errooooo"

def get_ac_status():
    return ac_status

def get_ac_current_temperature():
    return ac_current_temperature

def get_ac_desired_temperature():
    return ac_desired_temperature

def get_ac_mode():
    return ac_mode

def get_ac_fan_speed():
    return ac_fan_speed

def get_ac_error_message():
    return ac_error_message

def set_ac_desired_temperature(value):
    global ac_desired_temperature
    ac_desired_temperature = value
    write_value(files["ac_desired_temperature"], ac_desired_temperature)
    return ac_desired_temperature

def main():

    with open("/tmp/agent.log", 'a') as file:
        file.write(' '.join(sys.argv) + '\n')

    if len(sys.argv) < 3:
        log("Usage: agent.py <request-type> <MIB-oid> [<type> <value>]")
        print("Usage: agent.py <request-type> <MIB-oid> [<type> <value>]")
        return

    request_type = sys.argv[1]
    oid = sys.argv[2]

    if request_type == "-g":  # GET request
        if oid == ".1.3.6.1.3.99999.1.1.1":
            log(f"GET request for ac_status: {get_ac_status()}")
            print(".1.3.6.1.3.99999.1.1.1")
            print("integer")
            print(get_ac_status())
        elif oid == ".1.3.6.1.3.99999.1.1.2":
            log(f"GET request for ac_current_temperature: {get_ac_current_temperature()}")
            print(".1.3.6.1.3.99999.1.1.2")
            print("integer")
            print(get_ac_current_temperature())
        elif oid == ".1.3.6.1.3.99999.1.1.3":
            log(f"GET request for ac_desired_temperature: {get_ac_desired_temperature()}")
            print(".1.3.6.1.3.99999.1.1.3")
            print("integer")
            print(get_ac_desired_temperature())
        elif oid == ".1.3.6.1.3.99999.1.1.4":
            log(f"GET request for ac_mode: {get_ac_mode()}")
            print(".1.3.6.1.3.99999.1.1.4")
            print("integer")
            print(get_ac_mode())
        elif oid == ".1.3.6.1.3.99999.1.1.5":
            log(f"GET request for ac_fan_speed: {get_ac_fan_speed()}")
            print(".1.3.6.1.3.99999.1.1.5")
            print("integer")
            print(get_ac_fan_speed())
        elif oid == ".1.3.6.1.3.99999.1.1.6":
            log(f"GET request for ac_error_message: {get_ac_error_message()}")
            print(".1.3.6.1.3.99999.1.1.6")
            print("string")
            print(get_ac_error_message())
        else:
            log("Invalid OID for GET request")
            print("NONE")
    elif request_type == "-s":  # SET request
        if len(sys.argv) == 5:
            type = sys.argv[3]
            value = sys.argv[4]
            if oid == ".1.3.6.1.3.99999.1.1.3":
                #if type == "i":
                    try:
                        value = int(value)
                        result = set_ac_desired_temperature(value)
                        log(f"SET request for ac_desired_temperature: {result}")
                        print(".1.3.6.1.3.99999.1.1.3")
                        print("integer")
                        print(result)
                    except ValueError:
                        log("Invalid value for SET request")
                        print("Invalid value")
                #else:
                 #   log(f"Unsupported type for SET request: {type}")
                 #   log(f"jesus te amo")
                 #   log(sys.argv[3])
                 #   print(f"Unsupported type: {type}")
            else:
                log("Invalid OID or arguments for SET request")
                print("NONE")
        else:
            log("Usage: agent.py -s <MIB-oid> <type> <value>")
            print("Usage: agent.py -s <MIB-oid> <type> <value>")
    else:
        log("Invalid request type")
        print("NONE")

if __name__ == "__main__":
    main()


Overwriting /tmp/agent.py


Set WOrking, Mais opcoes do set (funcinanod tbm)

|
|
|
|
|
V

In [159]:
#!/usr/bin/env python3

import sys
import os

# Definindo o diretório de dados na pasta /tmp
data_dir = '/tmp'

# Caminhos dos arquivos para cada variável
files = {
    "ac_status": os.path.join(data_dir, 'ac_status.txt'),
    "ac_current_temperature": os.path.join(data_dir, 'ac_current_temperature.txt'),
    "ac_desired_temperature": os.path.join(data_dir, 'ac_desired_temperature.txt'),
    "ac_mode": os.path.join(data_dir, 'ac_mode.txt'),
    "ac_fan_speed": os.path.join(data_dir, 'ac_fan_speed.txt'),
    "ac_error_message": os.path.join(data_dir, 'ac_error_message.txt')
}

# Funções para ler e escrever valores em arquivos
def read_value(filename):
    try:
        with open(filename, 'r') as file:
            return file.readline().strip()
    except FileNotFoundError:
        return None

def write_value(filename, value):
    try:
        with open(filename, 'w') as file:
            file.write(str(value))
        log(f"Successfully wrote {value} to {filename}")
    except Exception as e:
        log(f"Error writing to {filename}: {e}")

# Função de log para escrever mensagens no arquivo de log
def log(message):
    with open("/tmp/agent_output.log", 'a') as log_file:
        log_file.write(message + '\n')

# Inicializando os valores ao carregar o script
ac_status = int(read_value(files["ac_status"]) or 1)
ac_current_temperature = int(read_value(files["ac_current_temperature"]) or 22)
ac_desired_temperature = int(read_value(files["ac_desired_temperature"]) or 24)
ac_mode = int(read_value(files["ac_mode"]) or 2)
ac_fan_speed = int(read_value(files["ac_fan_speed"]) or 3)
ac_error_message = read_value(files["ac_error_message"]) or "errooooo"

def get_ac_status():
    return ac_status

def get_ac_current_temperature():
    return ac_current_temperature

def get_ac_desired_temperature():
    return ac_desired_temperature

def get_ac_mode():
    return ac_mode

def get_ac_fan_speed():
    return ac_fan_speed

def get_ac_error_message():
    return ac_error_message

def set_ac_status(value):
    global ac_status
    ac_status = value
    write_value(files["ac_status"], ac_status)
    return ac_status

def set_ac_current_temperature(value):
    global ac_current_temperature
    ac_current_temperature = value
    write_value(files["ac_current_temperature"], ac_current_temperature)
    return ac_current_temperature

def set_ac_mode(value):
    global ac_mode
    ac_mode = value
    write_value(files["ac_mode"], ac_mode)
    return ac_mode

def set_ac_fan_speed(value):
    global ac_fan_speed
    ac_fan_speed = value
    write_value(files["ac_fan_speed"], ac_fan_speed)
    return ac_fan_speed

def set_ac_error_message(value):
    global ac_error_message
    ac_error_message = value
    write_value(files["ac_error_message"], ac_error_message)
    return ac_error_message

def set_ac_desired_temperature(value):
    global ac_desired_temperature
    ac_desired_temperature = value
    write_value(files["ac_desired_temperature"], ac_desired_temperature)
    return ac_desired_temperature

def main():

    with open("/tmp/agent.log", 'a') as file:
        file.write(' '.join(sys.argv) + '\n')

    if len(sys.argv) < 3:
        log("Usage: agent.py <request-type> <MIB-oid> [<type> <value>]")
        print("Usage: agent.py <request-type> <MIB-oid> [<type> <value>]")
        return

    request_type = sys.argv[1]
    oid = sys.argv[2]

    if request_type == "-g":  # GET request
        if oid == ".1.3.6.1.3.99999.1.1.0":
            log(f"GET request for ac_status: {get_ac_status()}")
            print(".1.3.6.1.3.99999.1.1.0")
            print("integer")
            print(get_ac_status())
        elif oid == ".1.3.6.1.3.99999.1.2.0":
            log(f"GET request for ac_current_temperature: {get_ac_current_temperature()}")
            print(".1.3.6.1.3.99999.1.2.0")
            print("integer")
            print(get_ac_current_temperature())
        elif oid == ".1.3.6.1.3.99999.1.3.0":
            log(f"GET request for ac_desired_temperature: {get_ac_desired_temperature()}")
            print(".1.3.6.1.3.99999.1.3.0")
            print("integer")
            print(get_ac_desired_temperature())
        elif oid == ".1.3.6.1.3.99999.1.4.0":
            log(f"GET request for ac_mode: {get_ac_mode()}")
            print(".1.3.6.1.3.99999.1.4.0")
            print("integer")
            print(get_ac_mode())
        elif oid == ".1.3.6.1.3.99999.1.5.0":
            log(f"GET request for ac_fan_speed: {get_ac_fan_speed()}")
            print(".1.3.6.1.3.99999.1.5.0")
            print("integer")
            print(get_ac_fan_speed())
        elif oid == ".1.3.6.1.3.99999.1.6.0":
            log(f"GET request for ac_error_message: {get_ac_error_message()}")
            print(".1.3.6.1.3.99999.1.6.0")
            print("string")
            print(get_ac_error_message())
        else:
            log("Invalid OID for GET request")
            print("NONE")
    elif request_type == "-s":  # SET request
        if len(sys.argv) == 5:
            typ = sys.argv[3]
            value = sys.argv[4]
            try:
                if oid == ".1.3.6.1.3.99999.1.1":
                    value = int(value)
                    result = set_ac_status(value)
                    log(f"SET request for ac_status: {result}")
                    print(".1.3.6.1.3.99999.1.1.0")
                    print("integer")
                    print(result)
                elif oid == ".1.3.6.1.3.99999.1.2":
                    value = int(value)
                    result = set_ac_current_temperature(value)
                    log(f"SET request for ac_current_temperature: {result}")
                    print(".1.3.6.1.3.99999.1.2.0")
                    print("integer")
                    print(result)
                elif oid == ".1.3.6.1.3.99999.1.3":
                    value = int(value)
                    result = set_ac_desired_temperature(value)
                    log(f"SET request for ac_desired_temperature: {result}")
                    print(".1.3.6.1.3.99999.1.3.0")
                    print("integer")
                    print(result)
                elif oid == ".1.3.6.1.3.99999.1.4":
                    value = int(value)
                    result = set_ac_mode(value)
                    log(f"SET request for ac_mode: {result}")
                    print(".1.3.6.1.3.99999.1.4.0")
                    print("integer")
                    print(result)
                elif oid == ".1.3.6.1.3.99999.1.5":
                    value = int(value)
                    result = set_ac_fan_speed(value)
                    log(f"SET request for ac_fan_speed: {result}")
                    print(".1.3.6.1.3.99999.1.5.0")
                    print("integer")
                    print(result)
                elif oid == ".1.3.6.1.3.99999.1.6":
                    result = set_ac_error_message(value)
                    log(f"SET request for ac_error_message: {result}")
                    print(".1.3.6.1.3.99999.1.6.0")
                    print("string")
                    print(result)
                else:
                    log("Invalid OID or arguments for SET request")
                    log(oid)
                    log(len(sys.argv))
                    print("NONE")
            except ValueError:
                log("Invalid value for SET request")
                print("Invalid value")
        else:
            log("Usage: agent.py -s <MIB-oid> <type> <value>")
            print("Usage: agent.py -s <MIB-oid> <type> <value>")
    elif request_type == "-n":  # GETNEXT request
        log("getnext!!")
        if oid == ".1.3.6.1.3.99999.1.1.0":
            log(f"GETNEXT request for ac_status: .1.3.6.1.3.99999.1.2.0")
            print(".1.3.6.1.3.99999.1.2.0")
            print("integer")
            print(get_ac_current_temperature())
        elif oid == ".1.3.6.1.3.99999.1.2.0":
            log(f"GETNEXT request for ac_current_temperature: .1.3.6.1.3.99999.1.3.0")
            print(".1.3.6.1.3.99999.1.3.0")
            print("integer")
            print(get_ac_desired_temperature())
        elif oid == ".1.3.6.1.3.99999.1.3.0":
            log(f"GETNEXT request for ac_desired_temperature: .1.3.6.1.3.99999.1.4.0")
            print(".1.3.6.1.3.99999.1.4.0")
            print("integer")
            print(get_ac_mode())
        elif oid == ".1.3.6.1.3.99999.1.4.0":
            log(f"GETNEXT request for ac_mode: .1.3.6.1.3.99999.1.5.0")
            print(".1.3.6.1.3.99999.1.5.0")
            print("integer")
            print(get_ac_fan_speed())
        elif oid == ".1.3.6.1.3.99999.1.5.0":
            log(f"GETNEXT request for ac_fan_speed: .1.3.6.1.3.99999.1.6.0")
            print(".1.3.6.1.3.99999.1.6.0")
            print("string")
            print(get_ac_error_message())
        else:
            log("Invalid OID for GETNEXT request")
            print("NONE")
    else:
        log("Invalid request type")
        print("NONE")

if __name__ == "__main__":
    main()


Usage: agent.py <request-type> <MIB-oid> [<type> <value>]


MIB

AIRCONDITIONER-MIB DEFINITIONS ::= BEGIN

IMPORTS
    MODULE-IDENTITY, OBJECT-TYPE, Integer32, enterprises FROM SNMPv2-SMI
    DisplayString FROM SNMPv2-TC;

airConditioner MODULE-IDENTITY
    LAST-UPDATED "202406250000Z"
    ORGANIZATION "Exemplo de Organização"
    CONTACT-INFO
        "Email: exemplo@contato.com"
    DESCRIPTION
        "MIB para monitorar e controlar um ar condicionado"
    ::= { experimental 99999 }

-- Objetos principais
acMainObjects OBJECT IDENTIFIER ::= { airConditioner 1 }

-- Status do ar condicionado
acStatus OBJECT-TYPE
    SYNTAX INTEGER {
        off(0),
        on(1)
    }
    MAX-ACCESS read-write
    STATUS current
    DESCRIPTION
        "O status atual do ar condicionado, onde off (0) significa desligado e on (1) significa ligado."
    ::= { acMainObjects 1 }

-- Temperatura atual
acCurrentTemperature OBJECT-TYPE
    SYNTAX Integer32 (-30..50)
    UNITS "Celsius"
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION
        "A temperatura atual medida pelo sensor do ar condicionado em graus Celsius."
    ::= { acMainObjects 2 }

-- Temperatura desejada
acDesiredTemperature OBJECT-TYPE
    SYNTAX Integer32 (-30..50)
    UNITS "Celsius"
    MAX-ACCESS read-write
    STATUS current
    DESCRIPTION
        "A temperatura desejada configurada para o ar condicionado em graus Celsius."
    ::= { acMainObjects 3 }

-- Modo de operação
acMode OBJECT-TYPE
    SYNTAX INTEGER {
        auto(1),
        cool(2),
        heat(3),
        fan(4)
    }
    MAX-ACCESS read-write
    STATUS current
    DESCRIPTION
        "O modo de operação do ar condicionado. auto (1), cool (2), heat (3), fan (4)."
    ::= { acMainObjects 4 }

-- Velocidade do ventilador
acFanSpeed OBJECT-TYPE
    SYNTAX INTEGER {
        low(1),
        medium(2),
        high(3)
    }
    MAX-ACCESS read-write
    STATUS current
    DESCRIPTION
        "A velocidade do ventilador do ar condicionado. low (1), medium (2), high (3)."
    ::= { acMainObjects 5 }

-- Mensagem de erro
acErrorMessage OBJECT-TYPE
    SYNTAX DisplayString (SIZE (0..255))
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION
        "Uma mensagem de erro, se houver, associada ao ar condicionado."
    ::= { acMainObjects 6 }

END

In [27]:
%%bash
snmpget -v2c -c public localhost .1.3.6.1.3.99999.1.1.0
snmpget -v2c -c public localhost .1.3.6.1.3.99999.1.2.0
snmpget -v2c -c public localhost .1.3.6.1.3.99999.1.3.0
snmpget -v2c -c public localhost .1.3.6.1.3.99999.1.6.0

snmpset -v2c -c private localhost .1.3.6.1.3.99999.1.3 i 122
snmpget -v2c -c public localhost .1.3.6.1.3.99999.1.3.0

snmpset -v2c -c private localhost .1.3.6.1.3.99999.1.6 s jerivaldo
snmpget -v2c -c public localhost .1.3.6.1.3.99999.1.6.0

snmpgetnext -v2c -c public localhost .1.3.6.1.3.99999.1.1.0

snmpgetnext -v2c -c public localhost .1.3.6.1.3.99999.1.4.0

snmpget -v2c -c public localhost .1.3.6.1.3.99999.1.7.1.1.3.2

snmpget -v2c -c public localhost .1.3.6.1.3.99999.1.7.1.1.3.1


SNMPv2-SMI::experimental.99999.1.1.0 = INTEGER: 1
SNMPv2-SMI::experimental.99999.1.2.0 = INTEGER: 22
SNMPv2-SMI::experimental.99999.1.3.0 = INTEGER: 122
SNMPv2-SMI::experimental.99999.1.6.0 = STRING: "jerivaldo"


SNMPv2-SMI::experimental.99999.1.3 = INTEGER: 122
SNMPv2-SMI::experimental.99999.1.3.0 = INTEGER: 122
SNMPv2-SMI::experimental.99999.1.6 = STRING: "jerivaldo"
SNMPv2-SMI::experimental.99999.1.6.0 = STRING: "jerivaldo"
SNMPv2-SMI::experimental.99999.1.2.0 = INTEGER: 22
SNMPv2-SMI::experimental.99999.1.5.0 = INTEGER: 3
SNMPv2-SMI::experimental.99999.1.7.1.1.3.2 = STRING: "2024-06-01"
SNMPv2-SMI::experimental.99999.1.7.1.1.3.1 = STRING: "2024-01-01"
